<br>


# Ripple clean up

In this notebook we will:

- Aggregate all ripple datasets into one;
- Find closest position datapoint;
- Do a Quality check: the closest datapoint must be inside a trial minimum and maximum timestamps, otherwise, it means the detected ripple occurred during rat transportation, before or after the behavioral session.
- Calculate position at the moment of SWR onset based on closest data points using the rat’s velocity within those two points. 

- Save single dataset into a .csv file.


### Imports

In [1]:
import os
import re
import glob
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
from ephys_utils import get_file_list
import matplotlib.pyplot as plt
%matplotlib inline

<br>

### Function definition

In [2]:
def find_closest_xy_trial_nr_outcome_run_type(value, position):
    
    """
    Find the closest x to value in the position file
    value, float - Reference value;
    position, DataFrame - Contains the x position data
    """
    
    closest_indices = position['timestamp'].sub(value).abs().sort_values().index.to_list()
    
    if not closest_indices:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan
    else:
        closest_index=closest_indices[0]
        closest_data = position.loc[closest_index, ['timestamp','x', 'y', 'trial_nr', 'outcome', 'run_type']]
        closest_data['i']=closest_index
        return closest_data
       
def remove_ripples_outside_trials(ripple, trial_timestamps):
    
    # Collect the minimum and maximum timestamps of the trial nr and session the ripple
    # was attributed too after finiding the closest position point
    
    trial = trial_timestamps[
        (trial_timestamps['session_code']==ripple['session_code']) &
        (trial_timestamps['trial_nr']==ripple['trial_nr'])]
    
    # Get min and max timestamps for trial of interest
    min_t = trial['min'].iloc[0]
    max_t = trial['max'].iloc[0]
    
    if max_t > ripple['start_time'] > min_t:
        
        ripple['trial_bound']=True
    else:
        ripple['trial_bound']=False
        
    return ripple
    
def calculate_xy_ripple_time_using_velocity(point1, point2, deltat_ripple):
    '''
    Using the two closests position points, calculate velocity at ripple time.
    Will be used to infer ripple time xy position.
    point1 occurs before point2.
    '''
    # Calculate the velocity between two closest points
    deltat = abs(point2.timestamp-point1.timestamp)
    deltax = abs(point2.x - point1.x)
    deltay = abs(point2.y - point1.y)
    vx = deltax / deltat
    vy =  deltay/ deltat
    
    # Get movement deltax and deltay from first point to location of ripple start time
    deltax_ripple = vx * deltat_ripple
    deltay_ripple = vy * deltat_ripple
    
    return point1['x']+deltax_ripple, point1['y']+deltay_ripple
     
def calculate_xy_at_ripple_time(ripple, position):
    '''
    Get the two closest/adjacent position points to the ripple start time 
    (we already have the closest but not both). 
    Based on these two points, we calculate the approximate x in which the rat was 
    at the start of ripple occurrence.
    '''
    session_position = position[position['session_code']==ripple['session_code']].sort_values(
        'timestamp')
    
    # Get previous position datapoint as 2nd point
    if ripple['start_time']==ripple['timestamp']:
        
        x_ripple= ripple['x']
        y_ripple =ripple['y']
        
    elif ripple['start_time']!=ripple['timestamp']:   
        
        if ripple['start_time'] < ripple['timestamp']:   
            # t1 < Start time < t2 (closest)    
            t1 = session_position.loc[ripple['i']-1, :]
            t2 = ripple[['timestamp', 'x', 'y']]
             
        elif ripple['start_time'] > ripple['timestamp']:  
            # Closest timestamp (t1) < Start time < Second point (t2)
            t1 = ripple[['timestamp', 'x', 'y']]
            t2 = session_position.loc[ripple['i']+1, :]
        
        deltat_ripple = abs(ripple['start_time']-t1['timestamp'])  
        x_ripple, y_ripple = calculate_xy_ripple_time_using_velocity(t1, t2, deltat_ripple) 
        
    ripple['x_ripple']=x_ripple
    ripple['y_ripple']=y_ripple
    
    return ripple

<br>

### Path definition

In [3]:
# Where data from all rats is stored
main_path = path ='/Volumes/Biló/EphysData/' 

In [4]:
# Which folder names to process
folders = [
# Magalhaes   
'MAGALHAES_DNMP11_20trials_20190126160731',
'MAGALHAES_DNMP12_20trials_20190127105419',
'MAGALHAES_DNMP13_10trials_20190128135858',
'MAGALHAES_DNMP13_10trials_20190128142936',
'MAGALHAES_DNMP14_3trials_20190128201036',
'MAGALHAES_DNMP14_6trials_20190128190601',
'MAGALHAES_DNMP14_11trials_20190128193951',
'MAGALHAES_DNMP15_20trials_20190129013902',
# Napoleão
'NAPOLEAO_DNMP22_20trials_20180504094832',
'NAPOLEAO_DNMP24_15trials_20180505150034',   
# Homero
'HOMERO_DNMP4_12trials_20191105174624',
'HOMERO_DNMP16_20trials_20191113131818',
'HOMERO_DNMP23_20trials_20191117150748'   
]

#### Process the datasets


In [5]:
all_ripples=[]
all_position=[]
all_cp_rois=[]
all_start_rois=[]
all_rw1_rois=[]
all_rw2_rois=[]

for f in folders:
    
    folder_path = os.path.join(main_path, f)
    # Get rat and session codes to add to dataframe
    session_code = re.search(r'trials_(\d*)', f).group(1)
    rat_code = re.search(r'([A-Z]*)_DNMP', f).group(1)[0:3]
    print(f)
    # Get necessary file paths
    ripple_path = sorted(glob.glob(os.path.join(folder_path, '*_ripples.csv')))[0]
    position_path = sorted(glob.glob(os.path.join(folder_path, 'Timestamped_position', '*_timestamped_position_df_clean.csv')))[0]
    cp_roi_path = sorted(glob.glob(os.path.join(folder_path,'ROIS', '*cp_roi_converted.csv')))[0]
    start_roi_path = sorted(glob.glob(os.path.join(folder_path,'ROIS', '*start_roi_converted.csv')))[0]
    rw1_roi_path = sorted(glob.glob(os.path.join(folder_path,'ROIS', '*rw1_roi_converted.csv')))[0]
    rw2_roi_path = sorted(glob.glob(os.path.join(folder_path,'ROIS', '*rw2_roi_converted.csv')))[0]
    
    # Read the data files
    ripple = pd.read_csv(ripple_path)
    position = pd.read_csv(position_path)
    cp_roi = pd.read_csv(cp_roi_path).drop(['Unnamed: 0'], axis=1)
    start_roi = pd.read_csv(start_roi_path).drop(['Unnamed: 0'], axis=1)
    rw1_roi = pd.read_csv(rw1_roi_path).drop(['Unnamed: 0'], axis=1)
    rw2_roi = pd.read_csv(rw2_roi_path).drop(['Unnamed: 0'], axis=1)
    
    # find closest position point to ripple
    closest = ripple['start_time'].apply(find_closest_xy_trial_nr_outcome_run_type, args=(position,))
    
    ripple_info = pd.concat([ripple, closest], axis=1)
    
    # Add rat conde and session code to each dataframe
    for df in [ripple_info, position, cp_roi, start_roi]:
        df['rat_code']=rat_code
        df['session_code']=session_code

    # Add data to list of all dataframes per type
    all_ripples.append(ripple_info)
    all_position.append(position)
    all_cp_rois.append(cp_roi)
    all_start_rois.append(start_roi)
    all_rw1_rois.append(rw1_roi)
    all_rw2_rois.append(rw2_roi)

MAGALHAES_DNMP11_20trials_20190126160731
MAGALHAES_DNMP12_20trials_20190127105419
MAGALHAES_DNMP13_10trials_20190128135858
MAGALHAES_DNMP13_10trials_20190128142936
MAGALHAES_DNMP14_3trials_20190128201036
MAGALHAES_DNMP14_6trials_20190128190601
MAGALHAES_DNMP14_11trials_20190128193951
MAGALHAES_DNMP15_20trials_20190129013902
NAPOLEAO_DNMP22_20trials_20180504094832
NAPOLEAO_DNMP24_15trials_20180505150034
HOMERO_DNMP4_12trials_20191105174624
HOMERO_DNMP16_20trials_20191113131818
HOMERO_DNMP23_20trials_20191117150748


#### Concatenate datasets into single dataframe

In [6]:
# Transform list of dataframes into a single dataframe
ripples = pd.concat(all_ripples)
position = pd.concat(all_position)
cp_rois = pd.concat(all_cp_rois)
start_rois=pd.concat(all_start_rois)
rw1_rois=pd.concat(all_rw1_rois)
rw2_rois=pd.concat(all_rw2_rois)

In [7]:
ripples.head()

,start_time,end_time,duration,mean_zscore,median_zscore,max_zscore,min_zscore,speed_at_start,speed_at_end,max_speed,...,mean_speed,timestamp,x,y,trial_nr,outcome,run_type,i,rat_code,session_code
0,2.6132,2.7108,0.0976,2.071229,1.798367,5.561986,0.005338,3.332296,1.277678,3.722466,...,2.455469,104.832166,15.7,0.294444,1.0,1.0,S,0,MAG,20190126160731
1,3.3248,3.3878,0.0630,1.322741,0.655081,4.407361,0.006510,1.048231,2.408410,3.594685,...,2.241490,104.832166,15.7,0.294444,1.0,1.0,S,0,MAG,20190126160731
2,21.1554,21.1856,0.0302,2.751821,2.806042,4.814654,0.001425,1.969377,1.766030,1.969377,...,1.523685,104.832166,15.7,0.294444,1.0,1.0,S,0,MAG,20190126160731
3,35.7206,35.7936,0.0730,1.386809,1.168084,3.363480,0.005457,1.110022,2.866298,2.866298,...,1.018537,104.832166,15.7,0.294444,1.0,1.0,S,0,MAG,20190126160731
4,51.4962,51.5506,0.0544,1.753471,1.265615,5.202559,0.020166,1.596523,1.212070,1.596523,...,1.411683,104.832166,15.7,0.294444,1.0,1.0,S,0,MAG,20190126160731


<br>



## Curate ripples
<br>

#### Remove ripples if start time occurrs before or after first and final timestamps of position dataset or during transport periods.
That means those ripples occurred before, after the behavioral session or between runs

In [36]:
trial_timestamps = position.groupby(['session_code', 'trial_nr']).timestamp.agg(['min', 'max']).reset_index()
trial_timestamps.iloc[110:120]

,session_code,trial_nr,min,max
110,20190128201036,22.0,329.322291,460.248525
111,20190129013902,1.0,35.519578,143.294771
112,20190129013902,2.0,145.279680,268.566515
113,20190129013902,3.0,270.451238,400.981030
114,20190129013902,4.0,403.295552,507.763072
115,20190129013902,5.0,509.679782,644.840538
116,20190129013902,6.0,647.120410,768.091059
117,20190129013902,7.0,770.571469,892.037286
118,20190129013902,8.0,894.387571,1024.719104
119,20190129013902,9.0,1028.554854,1156.472384


In [14]:
ripples = ripples.apply(remove_ripples_outside_trials, args=(trial_timestamps,), axis=1)
# How many ripples are trial bound? How many are not
ripples.groupby(['trial_bound'])['start_time'].count()

trial_bound
False      99
True     1617
Name: start_time, dtype: int64

In [ ]:
#Check if trial bound FALSE ripples are all in missing Sample arm

In [35]:
ripples_false = ripples[ripples.trial_bound == False]
ripples_false[ripples_false.session_code =='20190129013902']

,start_time,end_time,duration,mean_zscore,median_zscore,max_zscore,min_zscore,speed_at_start,speed_at_end,max_speed,...,timestamp,x,y,trial_nr,outcome,run_type,i,rat_code,session_code,trial_bound
0,15.9294,15.9956,0.0662,3.063459,3.032445,6.260978,0.007493,3.673886,3.213777,3.722982,...,35.519578,0.894583,2.9375,1.0,1.0,S,0,MAG,20190129013902,False
1,19.8570,19.9766,0.1196,2.638388,2.563533,7.446493,0.037656,1.654053,2.099291,2.103551,...,35.519578,0.894583,2.9375,1.0,1.0,S,0,MAG,20190129013902,False
2,20.5514,20.6224,0.0710,1.937209,1.721069,4.392099,0.008684,2.377269,2.174814,2.450196,...,35.519578,0.894583,2.9375,1.0,1.0,S,0,MAG,20190129013902,False
3,21.6374,21.6768,0.0394,1.664431,1.432908,3.026634,0.002701,2.749583,1.917907,2.749583,...,35.519578,0.894583,2.9375,1.0,1.0,S,0,MAG,20190129013902,False
4,22.4852,22.6412,0.1560,1.874462,1.995098,4.067412,0.003647,0.977374,0.395054,1.005186,...,35.519578,0.894583,2.9375,1.0,1.0,S,0,MAG,20190129013902,False
5,23.9324,23.9892,0.0568,2.086620,1.702122,4.491512,0.016461,0.562379,0.318649,0.625488,...,35.519578,0.894583,2.9375,1.0,1.0,S,0,MAG,20190129013902,False


In [23]:
t_diffs = ripples_false.timestamp - ripples.start_time
t_diffs.describe()

count    1125.000000
mean       -0.952900
std       325.098184
min     -2117.499204
25%      -100.268805
50%        -5.671734
75%        61.526300
max      1278.193914
dtype: float64

In [37]:
# Drop outbound ripples
ripples = ripples[ripples.trial_bound==True]

<br>


#### Get xy position at ripple start time

Get the two closest/adjacent position points to the ripple start time (we already have the closest but not both). Based on these two points, we calculate the approximate x in which the rat was at the start of ripple occurrence.

In [38]:
ripples = ripples.apply(calculate_xy_at_ripple_time, args=(position,), axis=1)

In [39]:
ripples = ripples[['start_time', 'end_time', 
         'trial_nr', 'outcome', 
         'run_type', 'rat_code', 
         'session_code', 'x_ripple', 'y_ripple']]

In [41]:
ripples.to_csv(os.path.join(main_path, 'clean_ripples.csv'), index=False)
position.to_csv(os.path.join(main_path, 'position.csv'), index=False)
cp_rois.to_csv(os.path.join(main_path, 'cp_rois.csv'), index=False)
start_rois.to_csv(os.path.join(main_path, 'start_rois.csv'), index=False)
rw1_rois.to_csv(os.path.join(main_path, 'rw1_rois.csv'), index=False)
rw2_rois.to_csv(os.path.join(main_path, 'rw2_rois.csv'), index=False)